In [1]:
%run ItemModel.ipynb
%run APIService.ipynb

In [2]:
import inspect, re
import pandas as pd
import csv

# Class DataCrawler

In [5]:
class DataCrawler(apiServiceDelegate):
    newest = 0
    keyword = "thoi trang nam"
    categoryids = "9572"
    items_json = []
    all_items = [] # List SHOPPEItem
    itemModel = SHOPPEItem()
    fileName = 'ShoppeItem.tsv'
 
    APIService = APIService(newest, keyword, categoryids)
    
    i = 1
    def __init__(self, newest, keyword, categoryids):
        self.newest = newest
        self.keyword = keyword
        self.categoryids = categoryids
        self.APIService = APIService(self.newest, self.keyword, self.categoryids)
        self.APIService.delegate = self
        pass
    
    def setNewest(self,newest):
        self.newest = newest
        pass
    
    def crawlData(self):
        while True:
            try:
                self.APIService.setNewest(self.newest)
                check = self.APIService.get()
                if check == False: break
                
            except Exception as e:
                self.parseData()
                print(f'Error: {e}')
                break
        pass
    
    def parseData(self):
        self.all_items.clear()
        for item in self.items_json:
            itemModel = SHOPPEItem()
            itemModel.name = item['name']
            itemModel.shop_location = item['shop_location']
            itemModel.ctime = item['ctime'] #time since 1970
            itemModel.time_now = int(time.time())
            itemModel.sell_time = itemModel.time_now - itemModel.ctime
            itemModel.historical_sold = item['historical_sold']
            itemModel.shopee_verified = item['shopee_verified']
            itemModel.discount = item['discount']
            itemModel.is_adult = item['is_adult']
            if len(item['tier_variations']) > 0:
                itemModel.options = 1
                for tier_variation in item['tier_variations']:
                    itemModel.options *= len(tier_variation['options'])
            else:
                itemModel.options = 0
            itemModel.show_official_shop_label_in_title = item['show_official_shop_label_in_title']
            itemModel.rating_star = item['item_rating']['rating_star']
            itemModel.rating_count = item['item_rating']['rating_count']
            itemModel.five_star = itemModel.rating_count[5]
            itemModel.four_star = itemModel.rating_count[4]
            itemModel.three_star = itemModel.rating_count[3]
            itemModel.two_star = itemModel.rating_count[2]
            itemModel.one_star = itemModel.rating_count[1]
            itemModel.flash_sale = item['flash_sale']
            itemModel.upcoming_flash_sale = item['upcoming_flash_sale']
            itemModel.price_min = item['price_min']
            itemModel.price_min_before_discount = item['price_min_before_discount']
            
            itemModel.price_max = item['price_max']
            itemModel.price_max_before_discount = item['price_max_before_discount']
            itemModel.price = item['price']
            itemModel.price_before_discount = item['price_before_discount']
            itemModel.coin_earn_label = item['coin_earn_label']
            itemModel.liked_count = item['liked_count']
            itemModel.view_count = item['view_count']
            itemModel.cmt_count = item['cmt_count']
            itemModel.is_preferred_plus_seller = item['is_preferred_plus_seller']
            itemModel.show_free_shipping = item['show_free_shipping']
            itemModel.images = len(item['images'])
            itemModel.video_info_list = len(item['video_info_list'])
            self.all_items.append(itemModel)
        pass
    
    def saveFile(self, name = fileName):
        self.fileName = name
        file = open(self.fileName, 'w', encoding='utf-8')
 
        file.write(f'category_id\tname\tshop_location\titem_public_time\titem_be_got_time\tsell_time\tshopee_verified\tdiscount\tis_adult\t'
                + f'options\tshow_official_shop_label_in_title\trating_star\tfive_star\tfour_star\tthree_star\ttwo_star\tone_star\t'
                + f'flash_sale\tupcoming_flash_sale\tprice_min\tprice_min_before_discount\t'
                + f'price_max\tprice_max_before_discount\tprice\tprice_before_discount\t'
                + f'coin_earn_label\tliked_count\tview_count\tcmt_count\t'
                + f'is_preferred_plus_seller\tshow_free_shipping\timages\tvideo_info_list\t'
                + f'historical_sold\n')

        for item in self.all_items:
                file.write(f'{self.categoryids}\t{item.name}\t{item.shop_location}\t{item.ctime}\t{item.time_now}\t{item.sell_time}\t{item.shopee_verified}\t{item.discount}\t{item.is_adult}\t'
                + f'{item.options}\t{item.show_official_shop_label_in_title}\t{item.rating_star}\t{item.five_star}\t'
                + f'{item.four_star}\t{item.three_star}\t{item.two_star}\t{item.one_star}\t'
                + f'{item.flash_sale}\t{item.upcoming_flash_sale}\t{item.price_min}\t{item.price_min_before_discount}\t'
                + f'{item.price_max}\t{item.price_max_before_discount}\t{item.price}\t{item.price_before_discount}\t'
                + f'{item.coin_earn_label}\t{item.liked_count}\t{item.view_count}\t{item.cmt_count}\t'
                + f'{item.is_preferred_plus_seller}\t{item.show_free_shipping}\t{item.images}\t{item.video_info_list}\t'
                + f'{item.historical_sold}\n')
                
    def updateDataWhenSuccess(self, json_pydata):
        print(len(json_pydata))
        print(f'get! {self.i}')
        
        if json_pydata['items'] != None:
            self.items_json.extend(json_pydata['items'])
        else:
            self.parseData()
            print('get All Data!')
            return False

        self.newest += 50
        self.i += 1
        time.sleep(10)
        return True
        pass
    def getDataError(self, error):
        self.parseData()
        print(f'Error when request: {error}')
        return False
        pass
    
    pass

# Crawling Data

In [6]:
crawler = DataCrawler(newest = 0, keyword = 'thoi trang nam', categoryids = '2829')

In [7]:
crawler.crawlData()


17
get! 1
17
get! 2
17
get! 3
17
get! 4
17
get! 5
17
get! 6
17
get! 7
17
get! 8
17
get! 9
17
get! 10
17
get! 11
17
get! 12
17
get! 13
17
get! 14
17
get! 15
17
get! 16
17
get! 17
17
get! 18
17
get! 19
17
get! 20
Error when request: Expecting value: line 1 column 1 (char 0)


In [82]:
# crawler.parseData()

In [8]:
# Check last request
#print(json.dumps(crawler.APIService.json_pydata, indent = 3, sort_keys=True))

In [22]:
# Check feature value
crawler.all_items[0].ctime

1602518724

In [9]:
# Test ParseData
print(crawler.all_items[0].name)
print(crawler.all_items[50].name)
print(crawler.all_items[1].name)
print(crawler.all_items[51].name)

Bộ Vest nam chính hãng Eric Tailor form đẹp lịch lãm chất liệu xuất dư xịn BSUITET1 - Leo Vatino
Áo Khoác Len Lông Cừu Dày Dặn Thời Trang Mùa Đông Cho Nam Giới
BỘ VEST NAM THIẾT KẾ SANG TRỌNG ĐẲNG CẤP
Áo khoác kiểu dáng đơn giản phong cách Hàn Quốc hợp thời trang cho nam/nữ


In [10]:
crawler.saveFile("../Data/[v3]ShoppeItem_thoiTrangNam_aoKhoacAoVest_0-949.tsv")

In [11]:
# Test saveFile
items_df = pd.read_csv('../Data/[v3]ShoppeItem_thoiTrangNam_aoKhoacAoVest_0-949.tsv', sep='\t')
pd.set_option('max_colwidth', 10000)

items_df



,category_id,name,shop_location,item_public_time,item_be_got_time,sell_time,shopee_verified,discount,is_adult,options,...,price_before_discount,coin_earn_label,liked_count,view_count,cmt_count,is_preferred_plus_seller,show_free_shipping,images,video_info_list,historical_sold
0,2829,Bộ Vest nam chính hãng Eric Tailor form đẹp lịch lãm chất liệu xuất dư xịn BSUITET1 - Leo Vatino,Hà Nội,1609905131,1610610053,704922,True,33%,False,20,...,120000000000,None,17,1419,1,False,False,9,0,2
1,2829,BỘ VEST NAM THIẾT KẾ SANG TRỌNG ĐẲNG CẤP,Hà Nội,1610477459,1610610053,132594,False,None,False,45,...,0,None,0,30,0,False,False,9,1,0
2,2829,"[ Mua Áo Tặng Quần ] Áo Vest nam Hàn Quốc Edana Menswear cao cấp không nhăn , không xù cho người 45 - 78 kg",Hà Nội,1603788552,1610610053,6821501,True,27%,False,20,...,75000000000,None,114,9285,69,False,False,9,1,112
3,2829,"Áo chống nắng nam 2020 , Chất Vải kim cương sần Siêu Mát, logo thêu",Hà Nội,1592064241,1610610053,18545812,True,46%,False,40,...,5900000000,None,33686,28709,1633,False,True,9,1,5493
4,2829,Áo Hoodie Trơn ❤️/Nam nữ unisex/FREESHIP 99K,TP. Hồ Chí Minh,1505555187,1610610053,105054866,False,47%,False,48,...,16000000000,None,5462,50435,1775,False,True,9,0,4512
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2829,áo phông Dota 2 in hình đẹp ⚡ FREESHIP ⚡Áo Game đẹp giá rẻ thời tran,Hà,2262440119,1610610053,-651830066,False,None,False,1,...,0,None,0,0,0,False,True,1,0,0
996,2829,Áo Khoác Jeans Thời Trang Nam Cao Cấ,TP. Hồ Chí,2676943861,1610610053,-1066333808,True,50%,False,3,...,58281734384,None,0,0,0,True,True,1,0,0
997,2829,Áo Khoác Phao Lót Bông Giữ Ấm Thời Tran,Nước n,2511440525,1610610053,-900830472,True,None,False,36,...,0,None,0,0,0,True,False,9,0,0
998,2829,Áo Khoác Giữ Ấm Dày Dặn Thời Trang Nam Trung,Nước n,1757807309,1610610053,-147197256,False,None,False,15,...,0,None,0,9,0,True,True,9,0,0


In [87]:
# Migrate Data
# Run this if you run the crawler before it've been updated to auto wirte the sell_time feature.

# items_df.insert(4, 'sell_time', items_df['item_be_got_time'] - items_df['item_public_time'])
# items_df.to_csv('ShoppeItem_thoiTrangNam_aoKhoacAoVest_0-949.tsv', sep = '\t', index=False)

In [88]:
del items_df